In [2]:
# Import Dependencies
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os

In [ ]:
# Define paths
train_dir = 'ASL_Alphabet_Dataset/asl_alphabet_train'
test_dir = 'ASL_Alphabet_Dataset/asl_alphabet_test'

batch_size = 64

# ImageDataGenerator for data augmentation and normalization 
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0, # Normalize the pixel values to be between 0 and 1
    shear_range=0.2, # Shear the image)
    zoom_range=0.2, # Zoom the image
    horizontal_flip=True, # Flip the image horizontally
    validation_split=0.2 # Split the data into training and validation sets
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training' # Use the training subset
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' # Use the validation subset
)




In [ ]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,
    verbose=1
)

In [ ]:
# Save the model
model.save('asl_model.keras')

In [ ]:

# Define the function to predict a single image
def predict_single_image(img_path, model, labels):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)
    predicted_label = labels[predicted_class_index[0]]
    
    return predicted_label

# Get the class labels
labels = (train_generator.class_indices)
labels = dict((v, k) for k, v in labels.items())

# Define the function to iterate through the folder and predict
def predict_folder(folder_path, model, labels):
    predictions = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            predicted_label = predict_single_image(img_path, model, labels)
            predictions.append((img_name, predicted_label))
    return predictions



# Get predictions
predictions = predict_folder(test_dir, model, labels)

# Print predictions
for img_name, predicted_label in predictions:
    print(f'Image: {img_name}, Predicted Label: {predicted_label}')